# Ch4 Convolution Neural Network

In [82]:
import torch
import numpy as np
from torch.autograd import Variable
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torchvision.datasets import mnist
%matplotlib inline

def data_tf(x):
    x = np.array(x, dtype='float32')
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    #x = x.reshape((-1,)) # 拉平
    x = torch.from_numpy(x)
    x = x.unsqueeze(0)
    return x

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [83]:
class CNN(nn.Module):
    def __init__ (self):
        super(CNN, self).__init__()
        self.c1=nn.Sequential(
            nn.Conv2d(1, 16 ,3),
            #nn.BatchNorm2d(16),
            nn.ReLU(True))
        
        self.c2=nn.Sequential(
        nn.Conv2d(16,32,kernel_size=3),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.c3=nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3),
            nn.ReLU(True)
            #nn.MaxPool2d(2,2)
        )
        
        self.c4=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        
        self.f1=nn.Sequential(
        nn.Linear(128*4*4,1024),
            nn.ReLU(True),
            nn.Linear(1024,128),
            nn.ReLU(True),
            nn.Linear(128,10)
        )
        
    def forward (self,x):
        x=self.c1(x)
        x=self.c2(x)
        x=self.c3(x)
        x=self.c4(x)
        x=x.view(x.size(0),-1)
        x=self.f1(x)
        return x

In [84]:
net=CNN()
# 定义 loss 函数
criterion = nn.CrossEntropyLoss()
optimzier = torch.optim.SGD(net.parameters(), 1e-3) # 使用随机梯度下降，学习率 0.01

In [86]:
losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    net.train()
    for im, label in train_data:
        im = Variable(im)
        label = Variable(label)
        # 前向传播
        out = net(im)
        loss = criterion(out, label)
        # 反向传播
        optimzier.zero_grad()
        loss.backward()
        optimzier.step()
        # 记录误差
        train_loss += loss.data[0]
        # 计算分类的准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().data[0]
        acc = num_correct / im.shape[0]
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
        # 在测试集上检验效果
    eval_loss = 0
    eval_acc = 0
    net.eval() # 将模型改为预测模式
    for im, label in test_data:
        im = Variable(im)
        label = Variable(label)
        out = net(im)
        loss = criterion(out, label)
        # 记录误差
        eval_loss += loss.data[0]
        # 记录准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().data[0]
        acc = num_correct / im.shape[0]
        eval_acc += acc
        
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(e, train_loss / len(train_data), train_acc / len(train_data), 
                     eval_loss / len(test_data), eval_acc / len(test_data)))

KeyboardInterrupt: 